In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/')
import os

# TODO: Fill in the Google Drive path where you uploaded the files 
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = './EECS545'
#GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab/final_project/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['COVID_combined', 'Normal', 'maskHE2_labels.csv', 'test', 'validation', 'ReadMe.gdoc', 'training.ipynb', 'training', 'train_dataset.npy', 'valid_dataset.npy', 'train_dataset_jiahong.npy', 'valid_dataset_jiahong.npy', 'Model_LungSegmentation', 'Model_Without_Lungseg', 'images_nomask', 'HE_output2.zip', 'Things to be done for Final Project.gdoc', 'Siamese_Learning.ipynb', 'grad_labels.csv', 'Matching_code', 'val_labels.csv', 'test_labels.csv', 'train_labels.csv', 'Copy of Siamese_Learning.ipynb', 'COVID-19 X-Ray Image Classification Using Transfer Learning and Contrastive Learning.gslides', 'Training_VGG16_CrossValidation_WithLungMasking.ipynb', 'grad_labels_4.csv', 'grad_labels_new.csv', 'create_label.ipynb']


In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

import time, os
os.environ["TZ"] = "US/Eastern"
time.tzset()

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch

import copy
from scipy.io import loadmat

**Transform dataset- Data Augmentation**

In [ ]:
FILES_PATH = GOOGLE_DRIVE_PATH + '/Model_Without_Lungseg/'
test_csv = os.path.join(GOOGLE_DRIVE_PATH,'test_labels.csv')
inceptionV3_test_results = loadmat(FILES_PATH + 'InceptionV3_Test_Index_nomask.mat')
vgg16_test_results = loadmat(FILES_PATH + 'VGG16_Test_Index_nomask.mat')
densenet121_test_results = loadmat(FILES_PATH + 'Densenet121_Test_Index_nomask.mat')

labels = pd.read_csv(test_csv)['encoded_labels'].values
inceptionV3 = inceptionV3_test_results['Result']
vgg16 = vgg16_test_results['Result'] 
densenet121 = densenet121_test_results['Result']

# densenet121

combined = np.concatenate((inceptionV3, vgg16, densenet121), axis = 1)
# np.size(concat, 0)
combined = combined.astype(int)

result = []
correct = 0
for i in range(labels.size):
  label = labels[i]
  # print(label)
  freq_label = np.bincount(combined[i]).argmax()
  result.append(freq_label)
  if label == freq_label:
    correct = correct + 1


print(float(correct/labels.size))

# densenet121 = densenet121.astype(int)
# correct = 0
# for i in range(labels.size):
#   label = labels[i]
#   # print(label)
#   d = densenet121[i][0]
#   if label == d:
#     correct = correct + 1


# print(float(correct/labels.size))

0.970954356846473


In [ ]:
import sklearn.metrics as sm
scores = sm.precision_recall_fscore_support(np.array(labels),np.array(result) , average='binary')
#print(scores)
precision = scores[0]
recall = scores[1]
fscore = scores[2]
print(fscore)
conf_matrix = sm.confusion_matrix(labels, result)
sensitivity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[1,0])
specificity = conf_matrix[1][1]/(conf_matrix[0,1]+conf_matrix[1,1])
print(sensitivity)
print(specificity)

0.97165991902834
0.9941860465116279
0.9498680738786279
